# Imports

In [1]:
import tensorflow as tf
print(f'tf.__version__: {tf.__version__}\n'
      f'tf.config.list_physical_devices("GPU"):\n'
      f'{tf.config.list_physical_devices("GPU")}', sep='')
from utils import load_and_preprocess_data
from vaec import VAEC
%load_ext tensorboard

tf.__version__: 2.10.1
tf.config.list_physical_devices("GPU"):
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Downloading and preprocessing data

In [2]:
x_train, y_train, x_test, y_test, mean_x_train, dev_x_train = \
    load_and_preprocess_data(split_ratio=0.7)

Data is loaded and preprocessed.
data_package.data.dtype: float64
data_package.data.shape: (569, 30)
data_package.target.dtype: int32
data_package.target.shape: (569,)
x_train.shape: (398, 30), y_train.shape: (398, 1)
x_test.shape: (171, 30), y_test.shape: (171, 1)


# Configuration for the network parts
- global_input_dim: For Wisconsin Breast Cancer Dataset it is 30.
- encoder_layers: List of neuron numbers in encoder MLP layers.
- decoder_layers: List of neuron numbers in decoder MLP layers.
- classifier_layers: List of neuron numbers in classifier_head MLP layers.
- latent_dim: Dimension of representation in autoencoder neck.
- activation: Any Keras built-in activation is usable.

In [3]:
config= {
    'global_input_dim' : 30,
    'encoder_layers' : [16, 8],
    'decoder_layers' : [8, 16],
    'latent_dim' : 8,
    'classifier_layers' : [8, 8, 4],
    'activation': 'elu',
    }

# Test case #1

A test case is created by instantiating VAEC with log_mark='LD08_fit_all_parts'. Then, training with fit_all_parts() member function is performed. LD08 stands for latent_dim=8.

In [4]:
vaec = VAEC(config=config, 
            x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test, 
            log_mark='LD08_fit_all_parts', summary=False)

vaec.fit_all_parts(epochs=2000)

Epoch 1/10
4/4 [==============================] - 2s 128ms/step - loss: 2.5511 - recon_loss: 1.2732 - kl_loss: 0.4667 - pred_loss: 0.6491 - accuracy: 0.5771 - val_loss: 2.5246 - val_recon_loss: 1.2452 - val_kl_loss: 0.4793 - val_pred_loss: 0.8001 - val_accuracy: 0.5380
Epoch 2/10
4/4 [==============================] - 0s 31ms/step - loss: 2.2673 - recon_loss: 1.0941 - kl_loss: 0.3234 - pred_loss: 0.5324 - accuracy: 0.5478 - val_loss: 2.3620 - val_recon_loss: 1.1829 - val_kl_loss: 0.3948 - val_pred_loss: 0.7843 - val_accuracy: 0.5088
Epoch 3/10
4/4 [==============================] - 0s 28ms/step - loss: 2.3092 - recon_loss: 1.1940 - kl_loss: 0.2708 - pred_loss: 0.8344 - accuracy: 0.5921 - val_loss: 2.1140 - val_recon_loss: 1.0310 - val_kl_loss: 0.3352 - val_pred_loss: 0.7478 - val_accuracy: 0.5848
Epoch 4/10
4/4 [==============================] - 0s 27ms/step - loss: 1.9401 - recon_loss: 0.7359 - kl_loss: 0.1674 - pred_loss: 0.6447 - accuracy: 0.5833 - val_loss: 2.0608 - val_recon_loss:

# Test case #2

Next test case is created by instantiating VAEC with log_mark='LD08_fit_ae_then_cl_head'. Then, first just encoder and decoder was trained as autoencoder, second the classifier_head is trained.

In [5]:
vaec = VAEC(config=config, 
            x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test, 
            log_mark='LD08_fit_ae_then_cl_head', summary=False)

vaec.fit_autoencoder(epochs=2000)
vaec.fit_classifier_head(epochs=2000)

Epoch 1/10
4/4 [==============================] - 1s 96ms/step - loss: 2.0911 - recon_loss: 1.2409 - kl_loss: 0.5913 - pred_loss: 0.7088 - accuracy: 0.3893 - val_loss: 1.6298 - val_recon_loss: 1.1633 - val_kl_loss: 0.4665 - val_pred_loss: 0.9609 - val_accuracy: 0.3860
Epoch 2/10
4/4 [==============================] - 0s 28ms/step - loss: 2.0002 - recon_loss: 1.6811 - kl_loss: 0.4614 - pred_loss: 1.1028 - accuracy: 0.3877 - val_loss: 1.4587 - val_recon_loss: 1.0723 - val_kl_loss: 0.3865 - val_pred_loss: 0.9402 - val_accuracy: 0.3918
Epoch 3/10
4/4 [==============================] - 0s 22ms/step - loss: 1.7135 - recon_loss: 1.2308 - kl_loss: 0.4485 - pred_loss: 0.9328 - accuracy: 0.3894 - val_loss: 1.3654 - val_recon_loss: 1.0401 - val_kl_loss: 0.3254 - val_pred_loss: 1.0491 - val_accuracy: 0.4269
Epoch 4/10
4/4 [==============================] - 0s 27ms/step - loss: 1.3706 - recon_loss: 0.8100 - kl_loss: 0.1951 - pred_loss: 0.9539 - accuracy: 0.4273 - val_loss: 1.2805 - val_recon_loss: 

# Test case #3

It is an extension of previous case, the difference is that now first pre-training is performed with fit_all_parts() member function. Then ancoder and decoder is trained with fit_autoencoder(), finally classifier head is traind separately with fit_classifier_head().

In [6]:
vaec = VAEC(config=config, 
            x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test, 
            log_mark='LD08_fit_all_ae_then_cl_head', summary=False)

vaec.fit_all_parts(epochs=2000)
vaec.fit_autoencoder(epochs=2000)
vaec.fit_classifier_head(epochs=2000)

Epoch 1/10
4/4 [==============================] - 1s 97ms/step - loss: 2.2832 - recon_loss: 1.0059 - kl_loss: 0.2891 - pred_loss: 0.7004 - accuracy: 0.5420 - val_loss: 2.1032 - val_recon_loss: 1.0993 - val_kl_loss: 0.3466 - val_pred_loss: 0.6573 - val_accuracy: 0.5789
Epoch 2/10
4/4 [==============================] - 0s 26ms/step - loss: 2.0835 - recon_loss: 1.1003 - kl_loss: 0.2622 - pred_loss: 0.4635 - accuracy: 0.6269 - val_loss: 2.0864 - val_recon_loss: 1.1585 - val_kl_loss: 0.3026 - val_pred_loss: 0.6253 - val_accuracy: 0.6608
Epoch 3/10
4/4 [==============================] - 0s 24ms/step - loss: 2.1879 - recon_loss: 1.2197 - kl_loss: 0.3569 - pred_loss: 0.6338 - accuracy: 0.6374 - val_loss: 1.9525 - val_recon_loss: 1.0843 - val_kl_loss: 0.2694 - val_pred_loss: 0.5989 - val_accuracy: 0.6433
Epoch 4/10
4/4 [==============================] - 0s 26ms/step - loss: 1.9191 - recon_loss: 0.9672 - kl_loss: 0.1853 - pred_loss: 0.5389 - accuracy: 0.7191 - val_loss: 1.8572 - val_recon_loss: 

# Test case #4

In this test the full-mlp classifier built up form encoder+classifier_head is trained directly.

In [7]:
vaec = VAEC(config=config, 
            x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test, 
            log_mark='LD08_fit_mlp_classifier', summary=False)

vaec.fit_mlp_classifier(epochs=2000)

Epoch 1/10
4/4 [==============================] - 1s 94ms/step - loss: 0.8092 - recon_loss: 1.8641 - kl_loss: 1.3089 - pred_loss: 0.7669 - accuracy: 0.3441 - val_loss: 0.7924 - val_recon_loss: 1.3993 - val_kl_loss: 0.8800 - val_pred_loss: 0.7924 - val_accuracy: 0.4327
Epoch 2/10
4/4 [==============================] - 0s 25ms/step - loss: 0.7821 - recon_loss: 1.2782 - kl_loss: 0.5572 - pred_loss: 0.8135 - accuracy: 0.3995 - val_loss: 0.6912 - val_recon_loss: 1.4653 - val_kl_loss: 0.8393 - val_pred_loss: 0.6912 - val_accuracy: 0.5439
Epoch 3/10
4/4 [==============================] - 0s 27ms/step - loss: 0.6964 - recon_loss: 1.6518 - kl_loss: 0.7249 - pred_loss: 0.6871 - accuracy: 0.5495 - val_loss: 0.6709 - val_recon_loss: 1.4791 - val_kl_loss: 0.8283 - val_pred_loss: 0.6709 - val_accuracy: 0.5906
Epoch 4/10
4/4 [==============================] - 0s 23ms/step - loss: 0.6711 - recon_loss: 1.9866 - kl_loss: 0.9012 - pred_loss: 0.7032 - accuracy: 0.6038 - val_loss: 0.6500 - val_recon_loss: 

In [8]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 15432), started 5:14:43 ago. (Use '!kill 15432' to kill it.)